SSL 证书导入

In [1]:
import pip_system_certs.wrapt_requests

建立 LLM 会话

In [2]:
from google import genai

with open(r".\api_key", "r", encoding="utf8") as reader: 
    api_key = reader.readline()
client = genai.Client(api_key=api_key)

model_name = "gemini-2.5-flash"

认证 github

In [3]:
from github.Repository import Repository, NotSet
from github.PullRequest import PullRequest

import github

with open(r".\github_token", "r", encoding="utf8") as reader:
    access_token = reader.readline()

auth = github.Auth.Token(access_token)

筛选 PR

In [4]:
from typing import Literal, List

def filtre(
        repo:Repository,
        pr_state:Literal["open", "close", "all"]=NotSet, # pr 状态
        sort:Literal["created", "updated", "pushed", "popularity"]=NotSet, # 结果排序方式
        direction:Literal["asc", "desc"]=NotSet, # 结果顺序
        base:str=NotSet, # 目标分支
        head:str=NotSet, # 来源分支/来源用户
        prid:int=None, # 指定 pr 编号
        author:str=None, # 指定 pr 发起人
        day_back:int=None, # 回溯指定天数内
    ) -> List[PullRequest]:
    results = repo.get_pulls(pr_state, sort, direction, base, head)
    if prid: results = [pr for pr in results if pr.number == prid]
    if author: results = [pr for pr in results if pr.user.login == author]
    if day_back:
        # datetime 范围内
        from datetime import datetime, timedelta, timezone
        now = datetime.now().replace(tzinfo=timezone.utc)
        start_from = now - timedelta(days=day_back) 
        results = [pr for pr in results if pr.created_at >= start_from]
    return results

In [ ]:
## https://github.com/{browser-use/browser-use}/pull/xxx.diff
repository = "browser-use/browser-use"

with github.Github(auth=auth) as g:
    repo = g.get_repo(repository)
    filtered = filtre(repo, "open", author="MagMueller", day_back=10)
    if not filtered: raise ValueError("No matching PR found")
    pr = filtered[0]; url = pr.diff_url

ValueError: No matching PR found

prompt 模板 需要调整

In [ ]:
# user_prompt = """Please briefly summarize the changes made to the code from the following diff:
# """
user_prompt = """请简短总结以下的 PR 描述和代码差异，并用数字编号输出差异条目：
PR 描述：
{desc}
代码差异：
{diffs}
"""

In [ ]:
from utils import file_io_utils as fiu

diffs = fiu.get_remote_contents(url)

response = client.models.generate_content(
    model=model_name, 
    contents=[user_prompt.format(desc=f"{pr.title}\n{pr.body}", diffs=diffs)]
)

In [ ]:
from IPython.display import Markdown, display

display(Markdown(response.text))

此 PR 引入了一个名为 `execute_js` 的通用 JavaScript 执行工具，该工具通过 Chrome DevTools Protocol (CDP) 的 `Runtime.evaluate` 方法执行任意 JavaScript 代码。它旨在解决现有工具无法处理的复杂浏览器交互和数据提取问题，例如坐标点击、批量表单填写、悬停、拖拽、链接抓取和页面探索。

代码差异：
1.  新增了 `ExecuteCDPAction` Pydantic 模型，其中包含一个 `js_code` 字段用于传递要执行的 JavaScript 代码。
2.  在 `browser_use/tools/service.py` 文件中导入了 `ExecuteCDPAction`。
3.  在 `service.py` 中注册了一个新的异步操作 `execute_js`。
4.  `execute_js` 操作获取一个 CDP 会话，并使用 `cdp_session.cdp_client.send.Runtime.evaluate` 执行提供的 `js_code`。
5.  它将 JavaScript 执行结果的 `value` 作为 `extracted_content` 返回，或在执行失败时返回清晰的错误信息。